In [1]:
%%bash
chia version
cdv --version

1.4.0
cdv, version 1.0.8


In [2]:
import sys
sys.path.insert(0, "../../../shared")
from utils import (load_program, print_program, print_puzzle)

from chia.types.blockchain_format.program import Program

counter_puzzle = load_program("counter.clsp", ["../../../shared"])
create_coin_puzzle =  load_program("create-coin.clsp", ["../../../shared"])
print(counter_puzzle.get_tree_hash())
print(create_coin_puzzle.get_tree_hash())

d5b0cadac424ef077dc8eacc449d0bb8544639cc39f5462a48a8d6b5750a4483
e016e18ebf275a5e0dfba0ba7d76c6969a727ef903de57a37386afef944fbe79


In [3]:
MOD = counter_puzzle
AMOUNT = 112
MAX_COUNT = 3
PUZZLE_HASH = 0x900ddeed
START_COUNT = 0
TERMINAL_PUZZLE = create_coin_puzzle.curry(PUZZLE_HASH, AMOUNT)
puzzle = counter_puzzle.curry(
    MOD,
    MAX_COUNT,
    TERMINAL_PUZZLE,
    AMOUNT,
    START_COUNT
)
print(puzzle.get_tree_hash())

2206e10bac429d1f751e36f8feefffcaf81590b85c8112178a91cabe424ba9c0


In [4]:
from clvm_tools.binutils import disassemble
def run_puzzle(puzzle):
    result = puzzle.run(Program.to([]))
    print_program(result)

print_puzzle(puzzle, tail=60)
run_puzzle(puzzle)
puzzle = counter_puzzle.curry(MOD, MAX_COUNT, TERMINAL_PUZZLE, AMOUNT, START_COUNT + 1)
print(puzzle.get_tree_hash())

print_puzzle(puzzle, 60)
run_puzzle(puzzle)
puzzle = counter_puzzle.curry(MOD, MAX_COUNT, TERMINAL_PUZZLE, AMOUNT, START_COUNT + 2)
print(puzzle.get_tree_hash())

print_puzzle(puzzle, 60)
run_puzzle(puzzle)
puzzle = counter_puzzle.curry(MOD, MAX_COUNT, TERMINAL_PUZZLE, AMOUNT, START_COUNT + 3)
print(puzzle.get_tree_hash())

print_puzzle(puzzle)
run_puzzle(puzzle)

... 0x00900ddeed) (c (q . 112) 1))) (c (q . 112) (c (q) 1))))))
((51 0x680fceade0f1b4b16e612d9b1d48ee90ceb0e32a3c1652be9c89ce4f05d7c4a3 112))
680fceade0f1b4b16e612d9b1d48ee90ceb0e32a3c1652be9c89ce4f05d7c4a3
...0900ddeed) (c (q . 112) 1))) (c (q . 112) (c (q . 1) 1))))))
((51 0x6217555e127f79e3cf9ffba852949a2d1a3b8fa69ca70805037e73ccadc2e59e 112))
6217555e127f79e3cf9ffba852949a2d1a3b8fa69ca70805037e73ccadc2e59e
...0900ddeed) (c (q . 112) 1))) (c (q . 112) (c (q . 2) 1))))))
((51 0x964c8ed7ec2aac80addcab109f0fb58d369634b526fbac08a5c42e39f9980ff5 112))
964c8ed7ec2aac80addcab109f0fb58d369634b526fbac08a5c42e39f9980ff5
(a (q 2 (q 2 (i (= 11 95) (q 2 23 ()) (q 2 12 (c 2 (c 5 (c 11 (c 23 (c 47 (c 95 ())))))))) 1) (c (q (51 4 (c 8 (c (a 30 (c 2 (c (a 10 (c 2 (c 5 (c (c 5 (c 11 (c 23 (c 47 (c (+ 95 (q . 1)) ()))))) ())))) ()))) (c 47 ()))) ()) (c (q . 2) (c (c (q . 1) 5) (c (a 22 (c 2 (c 11 (q 1)))) ()))) (a (i 5 (q 4 (q . 4) (c (c (q . 1) 9) (c (a 22 (c 2 (c 13 (c 11 ())))) ()))) (q . 11)) 1) 2

## Notes

### current count

The current count is embedded inside the puzzle as curried value. To get it, we could get the `puzzle_reveal` of the previous `coin_spend` from the blockchain and pull the value from the puzzle.

```clojure
(a ... (c (q . 42) 1))))))
```


In [5]:
puzzle = counter_puzzle.curry(MOD, MAX_COUNT, TERMINAL_PUZZLE, AMOUNT, 42)
count = puzzle.at("rrfrrfrrfrrfrrfrfr").as_int()
print(count)

42
